# kitti_2_las

Created by Luke McQuade, 25-July-2023.

Creates a set of georeferenced .las files from a sequence of KITTI .bin files (see https://github.com/PRBonn/semantic-kitti-api).

Designed for use in Google Colab.

## Usage

Upload the Kitti .bin files and poses.txt into the working directory, set the parameters, and run the remaining script. The .las files will be produced in the working directory.

## Warning

This is a work-in-progress; currently, world orientation (rotation) is not 100% working correctly. There are probably other issues.

## Parameters

In [60]:
# Number of files to process (we will look for files from 000001.bin to 000nnn.bin)
num_files = 10

# Coordinates of origin (world) and next track point, for determining position and orientation
origin_lon = 13.0354712 # degrees
origin_lat = 47.8158465 # degrees
origin_z = 418.992 # meters

next_lon = 13.0354742 # degrees
next_lat = 47.8158323 # degrees
next_z = 418.99 # meters

# Initial bearing (in radians). Manually specify this if known. Otherwise it is estimated from the above GPS points.
init_bearing_override = None

# Scale factor to convert input points to meters
SCALE_FACTOR = 0.1

# Target CRS EPSG (Expected: a projected CRS with meters as units of measurement)
OUT_EPSG = 31255 # (MGI / Austria GK Central)

In [44]:
%pip install laspy

In [61]:
import laspy
import numpy as np
from pyproj import CRS, Transformer
import math


# TODO: Fix this - leads to odd rotations later.
def calculate_bearing(x1, y1, x2, y2):
    """
    Calculates the bearing between two points (rectangular coordinates).
    Args:
        x1, y1: point 1 coordinates
        x2, y2: point 2 coordinates
    Returns:
        initial_compass_bearing: bearing in radians, clockwise from North.
    """
    dx = x2 - x1
    dy = y2 - y1
    bearing = np.arctan2(dy, dx)
    bearing = (bearing + 2*math.pi) % (2*math.pi)
    return bearing


def calculate_bearing_geo(pointA, pointB):
    """
    Calculates the bearing between two geographic points.
    Args:
        pointA: tuple with latitude and longitude of the first point.
        pointB: tuple with latitude and longitude of the second point.
    Returns:
        initial_compass_bearing: bearing in radians, clockwise from North.
    """
    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])
    diffLong = math.radians(pointB[1] - pointA[1])
    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))
    initial_bearing = math.atan2(x, y)
    initial_bearing = (initial_bearing + 2*math.pi) % (2*math.pi)
    return initial_bearing


def read_poses(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    poses = []
    for line in lines:
        pose = np.array(list(map(float, line.split()))).reshape(3, 4)

        # Complete the affine transform matrix
        pose = np.vstack([pose, [0, 0, 0, 1]])

        # In the input file, x is along-track and y is cross-track, but for later calculations, the opposite is assumed.
        # So, swap them here.
        old_dx = pose[0,3]
        pose[0,3] = pose[1,3]
        pose[1,3] = old_dx

        poses.append(pose)
    return poses


def process_bin_to_las(bin_path, pose, las_path, origin_proj, bearing):
    x1, y1, z1 = origin_proj
    # Load points from .bin file
    points_orig = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

    # Convert points to meters
    points = points_orig[:,:3] * SCALE_FACTOR  # 4th channel doesn't need conversion/transforming

    # Create homogeneous coordinates for points
    points_hom = np.ones((points.shape[0], 4))
    points_hom[:, :3] = points

    # Create a homogeneous transformation matrix for world coordinates
    homogeneous_transformation_world = np.array([
        [np.cos(-bearing), -np.sin(-bearing), 0, x1],
        [np.sin(-bearing),  np.cos(-bearing), 0, y1],
        [0, 0, 1, z1],
        [0, 0, 0, 1]
    ])

    # Create the combined transformation matrix
    combined_transformation = homogeneous_transformation_world @ pose

    # Apply the combined transformation to the points
    world_points_hom = (combined_transformation @ points_hom.T).T

    # Extract the world coordinates (discard the homogeneous coordinate)
    world_points = world_points_hom[:,:3]

    # Create las
    las = laspy.create(point_format=2, file_version='1.4')
    las.header.add_crs(CRS.from_user_input(OUT_EPSG))
    las.x = world_points[:, 0]
    las.y = world_points[:, 1]
    las.z = world_points[:, 2]
    las.intensity = points_orig[:, 3]  # instance id (stash it here as it's the same data type and the field is unused).
    las.write(las_path)

poses_path = "poses.txt"

poses = read_poses(poses_path)

# Transform origin to target CRS
t = Transformer.from_crs(f"EPSG:4326", f"EPSG:{OUT_EPSG}", always_xy=True)
origin_x, origin_y = t.transform(origin_lon, origin_lat)
next_x, next_y = t.transform(next_lon, next_lat)

if init_bearing_override is None:
  # TODO: Fix this - leads to odd rotations later.
  # init_bearing = calculate_bearing(origin_x, origin_y, next_x, next_y)
  init_bearing = calculate_bearing_geo((origin_lat, origin_lon), (next_lat, next_lon))

bin_paths = [f"{n:06}.bin" for n in range(1, num_files + 1)]
las_paths = [f"v1-{n:06}.las" for n in range(1, num_files + 1)]

for bin_path, las_path, pose in zip(bin_paths, las_paths, poses[:num_files]):
  print(f"Processing: {bin_path}, output: {las_path}")
  process_bin_to_las(bin_path, pose, las_path, (origin_x, origin_y, origin_z), init_bearing)

Processing: 000001.bin, output: v1-000001.las
Processing: 000002.bin, output: v1-000002.las
Processing: 000003.bin, output: v1-000003.las
Processing: 000004.bin, output: v1-000004.las
Processing: 000005.bin, output: v1-000005.las
Processing: 000006.bin, output: v1-000006.las
Processing: 000007.bin, output: v1-000007.las
Processing: 000008.bin, output: v1-000008.las
Processing: 000009.bin, output: v1-000009.las
Processing: 000010.bin, output: v1-000010.las
